In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

reader = SimpleDirectoryReader(input_dir="./files")
documents = reader.load_data()

parser = SentenceSplitter()
# nodes = parser.get_nodes_from_documents(documents)
# print(vars(nodes[0]))


In [14]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# for jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

# in-mem vector DB
import qdrant_client

client = qdrant_client.QdrantClient(location=":memory:")
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)

# Save the cache ( not working for me atm )
pipeline.cache.persist("./pipeline_cache/llama_cache.json")
pipeline.cache = IngestionCache.from_persist_path("./pipeline_cache/llama_cache.json")

nodes = pipeline.run(documents=documents)



100%|██████████| 3/3 [00:00<00:00,  4.62it/s]


In [15]:
# Create your index
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

# querying

In [22]:
from llama_index.llms.openrouter import OpenRouter
import os

llm = OpenRouter(
    api_key=os.environ['OPENROUTER_API_KEY'],
    max_tokens=8192,
    context_window=128000,
    model="qwen/qwen2.5-vl-72b-instruct:free",
)

query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query("what does commerce on the internet rely on")
print(response)

Commerce on the internet relies on a trust-based model that involves financial institutions mediating disputes. This model, however, has inherent weaknesses, such as the inability to conduct completely non-reversible transactions and the associated costs of mediation, which can limit the practicality of small transactions and introduce uncertainties. To address these issues, an electronic payment system based on cryptographic proof is proposed, which would allow direct transactions between parties without the need for a trusted third party, thereby protecting sellers from fraud and buyers through routine escrow mechanisms. This system aims to solve the double-spending problem through a peer-to-peer distributed timestamp server that generates computational proof of the chronological order of transactions.


In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator(llm=llm)
response = query_engine.query("who is the author of the attached paper?")
print(response)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

The author of the attached paper is not explicitly mentioned in the provided excerpts. The excerpts contain references and bibliographic information but do not include the author's name for the main document.


True